# Overview of PyTorch Autograd Engine

출처 : https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/

## 배경
Neural Network는 입력에 대해 실행되는 중첩(nested)된 함수들의 모음(collection)이다.  
PyTorch에서는 Tensor로 저장되는(Weight, bias로 구성된) 매개변수로 정의된다.

### 신경망 학습의 2단계
#### 1. 순전파(Forward Propagation)
순전파 단계에서, 신경망은 정답을 맞추기 위해 최선의 추측(best guess)을 한다. 이렇게 추측을 하기 위해서 입력 데이터를 각 함수들에서 실행합니다.

#### 2. 역전파(Backward Propagation)
역전파 단계에서, 신경망은 추측한 값에서 발생한 오류(error)에 비례하여(proportionate) 매개변수들을 적절히 조절(adjust)한다.  
출력(output)로부터 역방향으로 이동하면서 오류에 대한 함수들의 매개변수들의 미분값( 변화도(gradient) )을 수집하고, 경사하강법(gradient descent)을 사용하여 매개변수들을 최적화 한다.

## Autograd에서 미분(differentiation)


autograd 가 어떻게 변화도(gradient)를 수집하는지 살펴보자.  
requires_grad=True 를 갖는 2개의 텐서(tensor) a 와 b 를 만든다.   
requires_grad=True 는 autograd 에 모든 연산(operation)들을 추적해야 한다고 알려주자.  

In [8]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

이제 a 와 b 로부터 새로운 텐서 Q 를 만든다.

$$ Q = 3a^3 - b^2 $$

In [30]:
Q = 3*a**3 - b**2

a, b가 Neural Network의 매개변수 이고, Q가 오차라고 하면 아래 매개 변수들의 오차를 구해야 한다.  

$$ \frac{\partial Q}{\partial a} = 9a^2 $$  
$$ \frac{\partial Q}{\partial b} = 2b $$

Q 에 대해서 .backward() 를 호출할 때, autograd는 이러한 변화도들을 계산하고 이를 각 텐서의 .grad 속성(attribute)에 저장한다.  
Q 는 벡터(vector)이므로 Q.backward()에 gradient 인자(argument)를 명시적으로 전달해야 한다.  
gradient 는 Q 와 같은 모양(shape)의 텐서로, Q 자기 자신에 대한 변화도(gradient)를 나타낸다. 즉  

$$ \frac{dQ}{dQ} = 1 $$  

동일하게, Q.sum().backward() 와 같이 Q 를 스칼라(scalar) 값으로 집계(aggregate)한 뒤 암시적으로 .backward() 를 호출할 수도 있다.

In [10]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

In [11]:
# 수집된 변화도가 올바른지 확인
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


## 야코비안 행렬(Jacobian Matrix) 
수학적으로, 벡터 함수 $\vec{y}=f(\vec{x})$ 에서 $\vec{x}$ 에 대한 $\vec{y}$ 의 변화도는 야코비안 행렬(Jacobian Matrix) $J:$ 이다.

<img src = "images/1.png" width=400>

일반적으로, <code>torch.autograd</code> 는 벡터-야코비안 곱을 계산하는 엔진이다.  
이는, 주어진 어떤 벡터 $\vec{v}$ 에 대해 $J^{T} \cdot \vec{v}$ 을 연산한다.  
만약, $\vec v$가 스칼라 함수 $l = g(\vec v)$의 변화도(gradient)일 경우  

$$ \vec v = \Big( \frac{\partial l}{\partial y_1} \dots \frac{\partial l}{\partial y_m} \Big)^T $$

이며, 연쇄 법칙(chain rule)에 따라, 벡터-야코비안 곱은 $\vec{x}$ 에 대한 $l$ 의 변화도(gradient)가 된다.

<img src = "images/2.png" width=500>

위 에서 벡터-야코비안 곱의 이러한 특성을 사용했다. <code>external_grad</code> 가 $\vec{v}$를 뜻한다

## 예시

$$ f(x_1, x_2) = (log(x_1, x_2), sin(x_2)) $$

$$ g(y_1, y_2) = y_1y_2 $$

In [12]:
# 파이썬에서의 표현 예시
def f(x1,x2):
    a = x1*x2
    y1 = log(a)
    y2 = sin(x2)
    return (y1,y2)

연쇄함수와 도함수의 정의를 사용해 $ f(x_1, x_2) $를 손으로 유도하면
$$ y_1 편미분 $$
<br>
$$ \frac{\partial y_1}{\partial x_1} = \frac{\partial y_1}{\partial a} \frac{\partial a}{\partial x_1} = \frac{1}{\partial x_1}$$  

$$ \frac{\partial y_1}{\partial x_2} = \frac{\partial y_1}{\partial a} \frac{\partial a}{\partial x_2} = \frac{1}{\partial x_2}$$  



$$ y_2편미분 $$
<br>
$$ \frac{\partial y_2}{\partial x_1} = 0 $$
$$ \frac{\partial y_2}{\partial x_2} = cos(x_2) $$

스칼라 함수 $z = g(y_1, y_2)$에 대한 grdients는
$$ \frac{\partial z}{\partial y_1} = y_2 $$
$$ \frac{\partial z}{\partial y_2} = y_1 $$

야코비안 행렬로 표현하면  
$y_1 = log(x_1x_2)$  
$ y_2 = sin(x_2)$

<img src = "images/3.gif">

In [13]:
x1 = 0.5
x2 = 0.75
y1 = np.log(x1*x2)
y2 = np.sin(x2)

((y2/x1), y2/x2+np.cos(x2)*y1)

(1.3632775200466682, 0.19118983333660755)

$(x_1, x_2) = (0.5, 0.75)$ 일 때 $(\frac{dy}{x_1},\frac{dy}{x_2}) = (1.3633, 0.1912)$

#### torch에서 계산

In [29]:
x = torch.tensor([0.5, 0.75], requires_grad=True)
y = torch.log(x[0] * x[1]) * torch.sin(x[1])
y.backward()
x.grad

tensor([1.3633, 0.1912])

PyTorch 에서는 야코비엔 행렬을 직접 구성하지않고 
https://github.com/pytorch/pytorch/blob/master/tools/autograd/derivatives.yaml
에서 정의된 방식대로 벡터 곱을 적용하는 동안 뒤로 이동하는 연산 그래프를 만든다.

<img src = "images/4.png">

1. PyTorch가 작업을 실행할때마다 autograd engine은 backward할 graph를 구성한다. 

2. The reverse auto differentiation(역 자동 미분 모드)는 끝에 스칼라 변수를 추가하는 것으로 시작한다. 
(PyTorch에서 initial gradient는 사용자가 .backward()를 명시할 때 설정 된다. 기본 값 1)

3. forward가 완료 되면 그래프의 도함수가 실행되는 backward 에서 결과가 사용된다.  
(여기서 도함수들은 derivatives 'tools/autograd/derivatives.yaml' 파일에 저장된다.)

4. 최종 출력에서 도함수를 반복적으로 곱하여 input까지의 gradient가 chain rule에 따라 생성된다.

<img src = "images/5.png" width = 300>

#### Already calculated gradient 에서 시작(녹색)
$$\frac{\partial y}{\partial u} = 1$$ 
- z=w에 스칼라변수를 추가하면서 시작하는 초기 gradient 값
- tensor.backward의 parameter 에서 이 값을 설정 가능하다.  

#### backward function(빨간색)
$$\frac{\partial u}{\partial v}$$ 
- Derivatives.yaml에 등록되어 있다.
- 곱해지면 Already calculated gradient 가 된다.

#### next node(파랑색)
- 그래프의 다음 노드로 이동

### 정리
$ f(x_1, x_2) = (log(x_1, x_2), sin(x_2))$ 이 식에 그래프를 대입 해보면

- LogBackward node 를 보면 log의 도함수는 <code>grad.div(self.conj())</code>(켤레 복소수 계산 함수)로 이미 <code>derivatives.yaml</code> 안에 이미 계산 되어 있다.
- $(\frac{\partial z}{\partial y_1}\frac{1}{v})$는 위의 녹색, 빨간샌 사각형으로 볼 수 있다. 이 단계를 input gardient 까지 반복한다.
- $\frac{\partial z}{\partial x_2}$는 log grad만 반영되었으므로 sin grad를 더해주면 완료가된다.

### 즉 야코비안 행렬을 사용하지 않고 그래프를 사용하여도 gradient를 구할 수 있다.

### Tensor.backward()

<code>Tensor.backward(gradient=None, retain_graph=None, create_graph=False, inputs=None)</code>
- retain_graph 를 none으로 하면 gradient 계산 후 그래프가 사라진다.

In [15]:
a = torch.tensor([0.5, 0.75], requires_grad = True)
b = torch.log(a[0]*a[1])*torch.sin(a[1])
c = b.sum()
d = b.mean()

In [16]:
c.backward(retain_graph=True)
# c.backward()

In [17]:
d.backward()

## Autograd 구성요소
- tools /autograd : derivatives.yaml파일이 있다. derivatives(도함수?)를 정의한 내용을 볼 수 있다. C++코드를 생성하는데 사용된다.
https://github.com/pytorch/pytorch/tree/release/1.9/tools/autograd  
<br>  
- torch/autograd : Python에서 사용할 수 있는 autograd 요소가 있다.<code>torch.autograd.Function</code>에 따라 미분 가능 함수를 작성하는대 쓰는 클래스의 정의를 볼 수 있고 야코비안 행렬, hessian 관련 계산의 구성 요소를 보유  
https://github.com/pytorch/pytorch/tree/release/1.9/torch/autograd
<br>
- torch/csrc/autograd : 그래프 생성 및 실행 관련 코드가 있다. 극한의 성능이 요구되어 C++로 작성되어있다. <code>python_</code>로 시작된 이름을 가진 파일이 있고 Python의 autograd 엔진에 사용을 허가 한다는 의미이다.  
https://github.com/pytorch/pytorch/tree/release/1.9/torch/csrc/autograd
<br>  
- aten/ : "A Tensor Library"의 약자로 Tensor의 작업을 구현하는 C++라이브러리이다.

- c10/ : PyTorch의 에서 Tensor와 Storage데이터 구조의 실제 구현을 하는 기능들을 추상화(함수, 메소드 추출)하는 역할

## 연산 그래프의 생성 방식

이 모든 그래프의 tensor에서 gradient를 요구하는 Python 코드 부터 시작된다.  

<code>x = torch.tensor([0.5, 0.75], requires_grad=True)</code>  

<code>required_grad = True</code>가 설정(grad_fn)되면 c10에서 그래프 정보를 보유하는데 사용되는 <code>AutogradMeta</code>객체를 할당한다.

### Graphs, Nodes and Edges.
- 이제 생성한 Tensor(위에서의 x)를 사용하는 미분가능한 삼수를 호출 하면 <code>AutogradMeta</code>가 채워진다.
- Aten에서 torch 기능을 호출 하는 상황을 가정해보자. 

In [31]:
x = torch.tensor([0.5, 0.75], requires_grad=True)
y = torch.log(x[0] * x[1]) * torch.sin(x[1])

## C++ 생략
https://pytorch.org/blog/computational-graphs-constructed-in-pytorch/

### 그래프에 파이썬 함수 등록
Python에서 미분 가능한 함수를 정의하면 그래프에 포함된다.  
autograd python 정의 함수는 아래와 같다.

In [19]:
class Exp(torch.autograd.Function):
     @staticmethod
     def forward(ctx, i):
         result = i.exp()
         ctx.save_for_backward(result)
         return result

     @staticmethod
     def backward(ctx, grad_output):
         result, = ctx.saved_tensors
         return grad_output * result

# Call the function
Exp.apply(torch.tensor(0.5, requires_grad=True))
# Outputs: tensor(1.6487, grad_fn=<ExpBackward>)

tensor(1.6487, grad_fn=<ExpBackward>)

위 코드에서 autograd는 그래프를 생성할 파이썬 함수를 감지 했다. 

실제로는 <code>apply</code>는 <code>torch._C._FunctionBase c</code>에 정의되어 있지만 이 class는 파이썬소스에 있지 않다. <code>torch._C._FunctionBase c</code>는 Python C API를 사용하여 C++에 정의된다.